# Pre-Processing CSV data for MapAggC
#### General Approach:
1) Read in csv sheet of data.  Export fields into new dataframe.
2) Remove bad rows and elements.
3) Fix date datatype.
4) For ReportingUnit df, include ID value for easy match to shp file in ArcGIS (see below).
8) Export completed df as processed csZv.

#### Files Needed
1) dontopen_RU.csv, 
2) dontopen_AggLJAll.csv

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.options.display.max_rows  # Increases the length of a printed string from a DataFrame
pd.set_option('display.max_colwidth', -1)

C:\Users\rjame\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


## ReportingUnits Table and Export

#### General Notes
We drop the geometry field at this time.  API returns a encrypted version of it, which we can't work with at this time.  Have to instead match ReportingUnit_dim table to several separate shp files based on Unit Type (e.g., county, HUC8, custom, etc).  Have to build an ID value for this to work.  Each Unit Type file ID and match approach is different, see below.

#### ReportingUnit csv to ArcGIS shp file notes.
Creating ID to link p-csv to shp file works for every ReportUnitType but HUC8, as the ArcGIS shp we use doesn't incldue name in its data.  Have to cheat, and for all HUC8 types, the format should be "2_" + !HUC_8! + "_" + !StateNum!.  For the time, this was easier to do by hand than by code.

Wyoming also includes "... River Planning Basin" in its REportingUnitName.  Easier just to remove by hand.  Changed one WY shp file entry to Snake-Salt, from Salt-Snake.

California custom DAUCO also worked better using ReportingUnitNativeID.

AZ custom AM alsow worked better using ReportingUnitNative ID.  Shp didn't include a StateNum field, I just added it in.

USBR did work okay with ReportingUnitName.  Did have to update the shapefile with StateNum field = 100.

In [2]:
#Working Directory and Input File
workingDir = "C:/Users/rjame/Documents/WSWC Documents/Portal Creation Research"
os.chdir(workingDir)

In [3]:
RU_Input = "dontopen_RU.csv"
df_RU = pd.read_csv(RU_Input)
df_RU

,ReportingUnitID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,ReportingUnitUpdateDate,ReportingUnitProductVersion,StateCV,EPSGCodeCV,Geometry
0,249,test,test,test,County,1900-01-01,test,tt,test,NaN
1,250,test,test,test,County,1900-01-01,test,tt,test,NaN
2,254,00-01-03,00-01-03,Curlew Valley,Subarea,NaN,NaN,UT,EPSG:4326,0xE610000001043C000000C4648C3091255CC0175268181E00454023A1115A11255CC059C2B3780EFE44404C3ED088C1255CC080EDB92E0BFB44409AE530CBBC265CC03C558CBF4BFB4440F19AD105E2265CC015464A8248FC444035C7F839B0275CC04E107583F5F9444038CBA76A34265CC0F4BE00F50DF64440B1BB698F7A265CC002E5FD47BFF44440480DC8BDFB295CC0AE37BAB213F44440A47A4FBA742A5CC043D4B69C64F5444072A0DEB5D92B5CC0488CFC18A3F54440C55D48797C2C5CC069CE903DB7F44440D6F6DB193B2D5CC0DF46ADF0A7F2444030292232CC2B5CC0C42127055BF14440E4377DE95A2C5CC0616610E79DF0444062F38FE7A22C5CC094F96D957FED44408A6C7B42FA2D5CC027984E9F89EC44409ED86FA7592F5CC03DAFCB8B04E94440477FC2EE01315CC0F7E343181FE744407DCC9DCA49325CC0359BEDAAADE44440D88D9A659D325CC07850A6C8AFE24440CD8D8D7263345CC0E2ED6A41E8E14440F1E50A55BC355CC081DBEAB849DD444030EE4BE4DF355CC0B73A996578D94440FF21483451375CC0049804BE0DD94440299B841351375CC0FAEBB5EDC7D64440BFB31BD9ED385CC028722807C3D64440E3298F765A395CC07DA67E4EDFD54440E2C9438AD7385CC050C819CF63D544404210FBBE663B5CC030456E35CCD24440F9EBE212AD405CC0D67A6F314CD2444063C2008C0F425CC0AA5E102A8FD64440A37B27AF4A415CC0451EAA9AE5D844408B56DFDEC0415CC00A9AD3FCE5DA44400B9448A8A4405CC09541F6FCE2DA4440BACF9CA082415CC0E063E6BED9DC4440273D6F9000415CC049D77EECCCDD4440B55C3C9D3A415CC001071AB968DF44409629AD7FEC415CC0972AD734E7DF444091C03C9BCD415CC0329ECF8FA4E24440E487C2BF02415CC0F0811DEBCAE344405DB6984ADD405CC0A69A4B04C9E544405EF185B83E415CC08224976342E944402F92278BF5415CC0BD57D5FBE3EA444069FBE392A5415CC052B0072034EC4440DEC4DB793C435CC0CC67F98775EE444064CE76BDA7425CC0815596ACFCF04440B037886C10435CC028C86D16C8F34440ABD755923D445CC0016CF25447F64440095FBB3FC5455CC03DDCF8C5AEF74440A492EB8435475CC03F9D967B7CF64440AA437DB3894B5CC00685221C32F844407B07421AB94C5CC030FE448D08F84440055B9F36974E5CC0EB60B6F396F54440FC6DAD3844515CC084AC2C9AC4F54440986BA886594F5CC00B9BA7A01EF84440B3BD127A364F5CC06CC20F1FF6F944401C7CB7A50B4D5CC0943CA8402FFC44401B76B6C1604B5CC080F717E91A004540C4648C3091255CC0175268181E00454001000000020000000002000000FFFFFFFF0000000006000000000000000003
3,255,000-01-03,000-01-03,Clear Creek,Subarea,NaN,NaN,UT,EPSG:4326,0xE61000000104100000001B76B6C1604B5CC080F717E91A0045401C7CB7A50B4D5CC0943CA8402FFC444081F4F68E304F5CC01D28869801FA4440986BA886594F5CC00B9BA7A01EF84440A265166B9A505CC0AE03BD8D2BF7444029CDD83DBE505CC0FA7410BA24F64440863A445D12565CC06598EABE5FF3444053038E311C575CC06D5BDB0265F3444003597E7914585CC024F09BABDEF444401EE29D61A5595CC0F76F7101D1F34440280112FD455B5CC0CF750BE889F64440896D655B505D5CC0CDFEA3B63BF64440E8AF3D31FF5E5CC08EC4B44160F84440D2E0860E525E5CC0BF247DE27EFC444081EEF143D65C5CC060FB577CF9FF44401B76B6C1604B5CC080F717E91A00454001000000020000000002000000FFFFFFFF0000000006000000000000000003
4,256,00-07-02,00-07-02,Promontory Point,Subarea,NaN,NaN,UT,EPSG:4326,0xE610000001045600000033FF29DF4A175CC05816E60506004540C253DFF366185CC01AE3C0D87EFD444059300C1230175CC0C95406A390FA44401794B76006185CC0A5CF0E2CE7F74440825ED78E8C175CC0C7D6C8268EF64440E2FE6BE087185CC09501C0CD84F544407AF72111B2185CC0132F375BA4F24440EBE5EEA964195CC017CE920823F144407DF47C0AA0175CC0E37B48F9DDEC44401685F4BACF175CC01CEBFA3126EC444089476BCCEC165CC0048AA8FA0DEB4440341EB53112175CC055DA2C7D67E644402425E0195A165CC08E9B4C0EBEE444409A43AD8B00175CC000984389C3E34440DC3E2CD515165CC010E5092C04E144406430F1275D165CC09E00618B45DF4440BE5C00E9CF155CC0B8DBDE2275DD444058D73B71D3155CC060AF3AB214D844409755380711185CC0CAC32055DCD544400FF7936901185CC0F4057E0A8FD04440BAB61EE2FC185CC05BB5349515CF4440CD05FDE167195CC08EBFB607D5CA44404EF23C2993185CC0A553BFD569C74440CB82789E2C165CC04A65EBD439C244400B21FD22EB165CC03DFD11B30AC24440A1CDAA2EB9165CC0A970343E48BD44401D8A9EFB9F185CC0DFFF72A56FBC4440ADDF8839F91A5CC098C30

In [4]:
#removeing bad rows of df because they are null for some odd reason
df_RU = df_RU[(df_RU.ReportingUnitUUID != 'test')]

#Dropping fields we don't need.
df_RU = df_RU.drop(['Geometry', 'ReportingUnitUpdateDate', 'ReportingUnitProductVersion'], axis=1)

#Fixing Index
df_RU = df_RU.reset_index()
df_RU = df_RU.drop(columns=['index'])

df_RU

,ReportingUnitID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,StateCV,EPSGCodeCV
0,254,00-01-03,00-01-03,Curlew Valley,Subarea,UT,EPSG:4326
1,255,000-01-03,000-01-03,Clear Creek,Subarea,UT,EPSG:4326
2,256,00-07-02,00-07-02,Promontory Point,Subarea,UT,EPSG:4326
3,257,000-01-02,000-01-02,Yost,Subarea,UT,EPSG:4326
4,258,000-02-00,000-02-00,Goose Creek,Subarea,UT,EPSG:4326
...,...,...,...,...,...,...,...
1171,21044,AZag_RU1,Unspecified,PHOENIX AMA,Active Management Area,AZ,EPSG:4326
1172,21045,AZag_RU2,Unspecified,PINAL AMA,Active Management Area,AZ,EPSG:4326
1173,21046,AZag_RU3,Unspecified,PRESCOTT AMA,Active Management Area,AZ,EPSG:4326
1174,21047,AZag_RU4,Unspecified,SANTA CRUZ AMA,Active Management Area,AZ,EPSG:4326


In [5]:
#Creating Linking Element with Shape File.
#Format Ex: "1_" + !NAME! + "_" + !StateNum!

stateNumDic = {
    "UT" : "46",
    "NM" : "35",
    "NE" : "41",
    "CO" : "41",
    "WY" : "47",
    "CA" : "49",
    "AZ" : "48",
    "TX" : "37",
    "US" : "100"
}

def createTypeNameNum(colrowValueA, colrowValueB, colrowValueC):
    StringA = colrowValueA.strip()  # ReportingUnitTypeCV
    StringB = colrowValueB.strip().title()  # ReportingUnitName, in title case
    StringC = colrowValueC.strip()  # state for StateNumDic
    
    outStringA = ""
    
    if StringA == "County":
        outStringA = "1"
    if StringA == "HUC8":
        outStringA = "2"
    if StringA == "Basin":
        outStringA = "3"
    if StringA == "Detailed Analysis Unit by County (DAUCO)":
        outStringA = "4"
    if StringA == "Active Management Area":
        outStringA = "5"
    if StringA == "Tributary":
        outStringA = "6"
    if StringA == "Subarea":
        outStringA = "7"
    
    outList = outStringA + "_" + StringB + "_" + stateNumDic[StringC]
    return outList

df_RU['TypeNameNum'] = df_RU.apply(lambda row: createTypeNameNum(row['ReportingUnitTypeCV'], row['ReportingUnitName'], row['StateCV']), axis=1)
df_RU

,ReportingUnitID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,StateCV,EPSGCodeCV,TypeNameNum
0,254,00-01-03,00-01-03,Curlew Valley,Subarea,UT,EPSG:4326,7_Curlew Valley_46
1,255,000-01-03,000-01-03,Clear Creek,Subarea,UT,EPSG:4326,7_Clear Creek_46
2,256,00-07-02,00-07-02,Promontory Point,Subarea,UT,EPSG:4326,7_Promontory Point_46
3,257,000-01-02,000-01-02,Yost,Subarea,UT,EPSG:4326,7_Yost_46
4,258,000-02-00,000-02-00,Goose Creek,Subarea,UT,EPSG:4326,7_Goose Creek_46
...,...,...,...,...,...,...,...,...
1171,21044,AZag_RU1,Unspecified,PHOENIX AMA,Active Management Area,AZ,EPSG:4326,5_Phoenix Ama_48
1172,21045,AZag_RU2,Unspecified,PINAL AMA,Active Management Area,AZ,EPSG:4326,5_Pinal Ama_48
1173,21046,AZag_RU3,Unspecified,PRESCOTT AMA,Active Management Area,AZ,EPSG:4326,5_Prescott Ama_48
1174,21047,AZag_RU4,Unspecified,SANTA CRUZ AMA,Active Management Area,AZ,EPSG:4326,5_Santa Cruz Ama_48


In [6]:
#Creating Linking Element with Shape File.
#Format Ex: "1_" + !ID! + "_" + !StateNum!

stateNumDic = {
    "UT" : "46",
    "NM" : "35",
    "NE" : "41",
    "CO" : "42",
    "WY" : "47",
    "CA" : "49",
    "AZ" : "48",
    "TX" : "37",
    "US" : "100"
}

def createTypeNameNum(colrowValueA, colrowValueB, colrowValueC):
    StringA = colrowValueA.strip()  # ReportingUnitTypeCV
    StringB = colrowValueB.strip().title()  # ReportingUnitName, in title case
    StringC = colrowValueC.strip()  # state for StateNumDic
    
    outStringA = ""
    
    if StringA == "County":
        outStringA = "1"
    if StringA == "HUC8":
        outStringA = "2"
    if StringA == "Basin":
        outStringA = "3"
    if StringA == "Detailed Analysis Unit by County (DAUCO)":
        outStringA = "4"
    if StringA == "Active Management Area":
        outStringA = "5"
    if StringA == "Tributary":
        outStringA = "6"
    if StringA == "Subarea":
        outStringA = "7"
    
    outList = outStringA + "_" + StringB + "_" + stateNumDic[StringC]
    return outList

df_RU['TypeIDNum'] = df_RU.apply(lambda row: createTypeNameNum(row['ReportingUnitTypeCV'], row['ReportingUnitNativeID'], row['StateCV']), axis=1)
df_RU

,ReportingUnitID,ReportingUnitUUID,ReportingUnitNativeID,ReportingUnitName,ReportingUnitTypeCV,StateCV,EPSGCodeCV,TypeNameNum,TypeIDNum
0,254,00-01-03,00-01-03,Curlew Valley,Subarea,UT,EPSG:4326,7_Curlew Valley_46,7_00-01-03_46
1,255,000-01-03,000-01-03,Clear Creek,Subarea,UT,EPSG:4326,7_Clear Creek_46,7_000-01-03_46
2,256,00-07-02,00-07-02,Promontory Point,Subarea,UT,EPSG:4326,7_Promontory Point_46,7_00-07-02_46
3,257,000-01-02,000-01-02,Yost,Subarea,UT,EPSG:4326,7_Yost_46,7_000-01-02_46
4,258,000-02-00,000-02-00,Goose Creek,Subarea,UT,EPSG:4326,7_Goose Creek_46,7_000-02-00_46
...,...,...,...,...,...,...,...,...,...
1171,21044,AZag_RU1,Unspecified,PHOENIX AMA,Active Management Area,AZ,EPSG:4326,5_Phoenix Ama_48,5_Unspecified_48
1172,21045,AZag_RU2,Unspecified,PINAL AMA,Active Management Area,AZ,EPSG:4326,5_Pinal Ama_48,5_Unspecified_48
1173,21046,AZag_RU3,Unspecified,PRESCOTT AMA,Active Management Area,AZ,EPSG:4326,5_Prescott Ama_48,5_Unspecified_48
1174,21047,AZag_RU4,Unspecified,SANTA CRUZ AMA,Active Management Area,AZ,EPSG:4326,5_Santa Cruz Ama_48,5_Unspecified_48


In [7]:
#The Ouput
df_RU.to_csv('Pagg_ReportingUnit.csv', index=False)

## AggregatedAmounts_withOrg

In [8]:
AAwO_Input = "dontopen_AggLJAll.csv"
df_AAwO = pd.read_csv(AAwO_Input)
df_AAwO.head(5)

C:\Users\rjame\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,AggregatedAmountID,OrganizationID,ReportingUnitID,VariableSpecificID,WaterSourceID,MethodID,ReportYearCV,Amount,TimeframeStartID,TimeframeEndID,ReportingUnitUUID,ReportingUnitTypeCV,TimeframeStartID.1,TimeframeEndID.1,WaterSourceTypeCV,VariableCV,VariableSpecificCV,ApplicableResourceTypeCV,MethodTypeCV,State,BeneficialUseCV
0,15223,3,254,62,338412,36,2004,10.7,41.0,35.0,00-01-03,Subarea,41.0,35.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial
1,15224,3,254,62,338412,36,2005,3.0,29.0,44.0,00-01-03,Subarea,29.0,44.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial
2,15225,3,254,62,338412,36,2006,3.0,6.0,28.0,00-01-03,Subarea,6.0,28.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial
3,15226,3,254,62,338412,36,2007,3.0,33.0,26.0,00-01-03,Subarea,33.0,26.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial
4,15227,3,254,62,338412,36,2008,3.0,27.0,40.0,00-01-03,Subarea,27.0,40.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial


In [9]:
date_Input = "dontopen_Date_dim.csv"
df_date = pd.read_csv(date_Input)
df_date.head(5)

,DateID,Date,Year
0,1,2013-10-01,2013
1,2,2003-09-01,2003
2,3,2003-10-01,2003
3,4,2006-08-01,2006
4,5,2006-09-01,2006


In [10]:
#Retreiving TimeframeStart from Dates_dim Table
TimeframeStartdict = pd.Series(df_date.Date.values, index = df_date.DateID).to_dict()

# For creating TimeframeStart
def retrieveTimeframeStart(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        val1 = colrowValue
        try:
            outList = TimeframeStartdict[val1]
        except:
            outList = colrowValue
    return outList

df_AAwO['TimeframeStart'] = df_AAwO.apply(lambda row: retrieveTimeframeStart(row['TimeframeStartID']), axis=1)
df_AAwO.head(5)

,AggregatedAmountID,OrganizationID,ReportingUnitID,VariableSpecificID,WaterSourceID,MethodID,ReportYearCV,Amount,TimeframeStartID,TimeframeEndID,ReportingUnitUUID,ReportingUnitTypeCV,TimeframeStartID.1,TimeframeEndID.1,WaterSourceTypeCV,VariableCV,VariableSpecificCV,ApplicableResourceTypeCV,MethodTypeCV,State,BeneficialUseCV,TimeframeStart
0,15223,3,254,62,338412,36,2004,10.7,41.0,35.0,00-01-03,Subarea,41.0,35.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2004-10-01
1,15224,3,254,62,338412,36,2005,3.0,29.0,44.0,00-01-03,Subarea,29.0,44.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2005-10-01
2,15225,3,254,62,338412,36,2006,3.0,6.0,28.0,00-01-03,Subarea,6.0,28.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2006-10-01
3,15226,3,254,62,338412,36,2007,3.0,33.0,26.0,00-01-03,Subarea,33.0,26.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2007-10-01
4,15227,3,254,62,338412,36,2008,3.0,27.0,40.0,00-01-03,Subarea,27.0,40.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2008-10-01


In [11]:
#Retreiving TimeframeEnd from Dates_dim Table
TimeframeEnddict = pd.Series(df_date.Date.values, index = df_date.DateID).to_dict()

# For creating TimeframeEnd
def retrieveTimeframeEnd(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        val1 = colrowValue
        try:
            outList = TimeframeEnddict[val1]
        except:
            outList = colrowValue
    return outList

df_AAwO['TimeframeEnd'] = df_AAwO.apply(lambda row: retrieveTimeframeEnd(row['TimeframeEndID']), axis=1)
df_AAwO.head(5)

,AggregatedAmountID,OrganizationID,ReportingUnitID,VariableSpecificID,WaterSourceID,MethodID,ReportYearCV,Amount,TimeframeStartID,TimeframeEndID,ReportingUnitUUID,ReportingUnitTypeCV,TimeframeStartID.1,TimeframeEndID.1,WaterSourceTypeCV,VariableCV,VariableSpecificCV,ApplicableResourceTypeCV,MethodTypeCV,State,BeneficialUseCV,TimeframeStart,TimeframeEnd
0,15223,3,254,62,338412,36,2004,10.7,41.0,35.0,00-01-03,Subarea,41.0,35.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2004-10-01,2004-09-30
1,15224,3,254,62,338412,36,2005,3.0,29.0,44.0,00-01-03,Subarea,29.0,44.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2005-10-01,2005-09-30
2,15225,3,254,62,338412,36,2006,3.0,6.0,28.0,00-01-03,Subarea,6.0,28.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2006-10-01,2006-09-30
3,15226,3,254,62,338412,36,2007,3.0,33.0,26.0,00-01-03,Subarea,33.0,26.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2007-10-01,2007-09-30
4,15227,3,254,62,338412,36,2008,3.0,27.0,40.0,00-01-03,Subarea,27.0,40.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2008-10-01,2008-09-30


In [12]:
#Dropping fields we don't need.
df_AAwO = df_AAwO.drop(['TimeframeStartID', 'TimeframeEndID'], axis=1)
df_AAwO.head(5)

,AggregatedAmountID,OrganizationID,ReportingUnitID,VariableSpecificID,WaterSourceID,MethodID,ReportYearCV,Amount,ReportingUnitUUID,ReportingUnitTypeCV,TimeframeStartID.1,TimeframeEndID.1,WaterSourceTypeCV,VariableCV,VariableSpecificCV,ApplicableResourceTypeCV,MethodTypeCV,State,BeneficialUseCV,TimeframeStart,TimeframeEnd
0,15223,3,254,62,338412,36,2004,10.7,00-01-03,Subarea,41.0,35.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2004-10-01,2004-09-30
1,15224,3,254,62,338412,36,2005,3.0,00-01-03,Subarea,29.0,44.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2005-10-01,2005-09-30
2,15225,3,254,62,338412,36,2006,3.0,00-01-03,Subarea,6.0,28.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2006-10-01,2006-09-30
3,15226,3,254,62,338412,36,2007,3.0,00-01-03,Subarea,33.0,26.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2007-10-01,2007-09-30
4,15227,3,254,62,338412,36,2008,3.0,00-01-03,Subarea,27.0,40.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2008-10-01,2008-09-30


In [13]:
df_AAwO['BeneficialUseCV'].fillna('Unspecified', inplace=True)
df_AAwO

,AggregatedAmountID,OrganizationID,ReportingUnitID,VariableSpecificID,WaterSourceID,MethodID,ReportYearCV,Amount,ReportingUnitUUID,ReportingUnitTypeCV,TimeframeStartID.1,TimeframeEndID.1,WaterSourceTypeCV,VariableCV,VariableSpecificCV,ApplicableResourceTypeCV,MethodTypeCV,State,BeneficialUseCV,TimeframeStart,TimeframeEnd
0,15223,3,254,62,338412,36,2004,10.7,00-01-03,Subarea,41.0,35.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2004-10-01,2004-09-30
1,15224,3,254,62,338412,36,2005,3.0,00-01-03,Subarea,29.0,44.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2005-10-01,2005-09-30
2,15225,3,254,62,338412,36,2006,3.0,00-01-03,Subarea,6.0,28.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2006-10-01,2006-09-30
3,15226,3,254,62,338412,36,2007,3.0,00-01-03,Subarea,33.0,26.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2007-10-01,2007-09-30
4,15227,3,254,62,338412,36,2008,3.0,00-01-03,Subarea,27.0,40.0,Unspecified,Withdrawal,"Withdrawal, Public Supply",Modeled,Modeled,UT,Municipal/Industrial,2008-10-01,2008-09-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113747,452428,54,21048,146,565458,74,1996,0.0,AZag_RU5,Active Management Area,NaN,NaN,Surface Water,Supply,Supply,Surface Ground Water,Modeled,AZ,Unspecified,,
113748,452436,54,21048,146,565458,74,1997,25095.0,AZag_RU5,Active Management Area,NaN,NaN,Surface Water,Supply,Supply,Surface Ground Water,Modeled,AZ,Unspecified,,
113749,452437,54,21048,146,565458,74,1997,0.0,AZag_RU5,Active Management Area,NaN,NaN,Surface Water,Supply,Supply,Surface Ground Water,Modeled,AZ,Unspecified,,
113750,452444,54,21048,146,565458,74,1998,22924.0,AZag_RU5,Active Management Area,NaN,NaN,Surface Water,Supply,Supply,Surface Ground Water,Modeled,AZ,Unspecified,,


In [14]:
#The Ouput
df_AAwO.to_csv('Pagg_AggregatedAmountsAll.csv', index=False)